# IMPORT

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
from qutip import *
from qutip.qip.operations import rz
from qutip.qip.operations import cphase
from scipy.optimize import minimize, minimize_scalar

In [3]:
# 可交互图像
%matplotlib notebook
%matplotlib inline

# 参数设置

In [4]:
anharm = {'q0' : -2 * np.pi * 201 * 1e-3, 'c0' : -2 * np.pi * 200 * 1e-3, 'q1' : -2 * np.pi * 204 * 1e-3, 'c1' : -2 * np.pi * 206 * 1e-3, 'q2' : -2 * np.pi * 208 * 1e-3}
C_q = [97, 99, 97]
C_c = [95, 100]
C_ic = {'q0c0' : 3.8, 'q1c0' : 4.1, 'q1c1' : 4.3, 'q2c1' : 4}
C_12 = {'q0q1' : 0.27, 'q1q2' : 0.15}

g_qc = dict()
for k in C_ic:
    q, c = int(k[1]), int(k[3])
    g_qc[k] = 0.5 * C_ic[k] / np.sqrt(C_q[q] * C_c[c])

g_qq = dict()
for k in C_12:
    q0, q1 = int(k[1]), int(k[3])
    C_1cC_2c = 1
    for kk in C_ic:
        if ('q' + str(q0) in kk and 'q' + str(q1) + kk[2:] in C_ic) or ('q' + str(q1) in kk and 'q' + str(q0) + kk[2:] in C_ic):
            C_1cC_2c *= C_ic[kk]
            ck = int(kk[3])
    g_qq[k] = 0.5 * (1 + C_1cC_2c / (C_12[k] * C_c[ck])) * C_12[k] / np.sqrt(C_q[q0] * C_q[q1])

print(g_qq, g_qc)

{'q0q1': 0.002214401002636834, 'q1q2': 0.0016429426793757155} {'q0c0': 0.019792740520695653, 'q1c0': 0.021138514350399654, 'q1c1': 0.02160831302807306, 'q2c1': 0.02030692330267238}


# 定义系统哈密顿量

一个把算符张起来的函数

In [50]:
def tensorOperator(energyLevel, op, qubit, qubitNum):
    I = qeye(energyLevel)
    for dim in range(qubitNum):
        if dim == qubit:
            tempOp = op
        else:
            tempOp = I
        if dim == 0:
            tensorOp = tempOp
        else:
            tensorOp = tensor(tempOp, tensorOp)
    return tensorOp

哈密顿量

态的各个bit位：$|...q3c2q2c1q1c0q0\rangle$,输入频率时按照$[\omega_{q0},\omega_{q1},\cdots]$

In [51]:
def H(g_qc, g_qq, omegaq, omegac, qNum, cNum):
    noH = True
    for freQ in range(len(omegaq)):
        if noH:
            H0 = omegaq[freQ] * aqDagList[freQ] * aqList[freQ] + 0.5 * anharm['q' + str(freQ)] * aqDagList[freQ] * aqDagList[freQ] * aqList[freQ] * aqList[freQ]
            noH = False
        else:
            H0 += omegaq[freQ] * aqDagList[freQ] * aqList[freQ] + 0.5 * anharm['q' + str(freQ)] * aqDagList[freQ] * aqDagList[freQ] * aqList[freQ] * aqList[freQ]
    for freC in range(len(omegac)):
        H0 += omegac[freC] * acDagList[freC] * acList[freC] + 0.5 * anharm['c' + str(freC)] * acDagList[freC] * acDagList[freC] * acList[freC] * acList[freC]
    noH = True
    for g in g_qc:
        q, c = int(g[1]), int(g[3])
        if q < qNum and c < cNum:
            if noH:
                Hi = g_qc[g] * np.sqrt(omegaq[q] * omegac[c]) * (aqDagList[q] + aqList[q]) * (acList[c] + acDagList[c])
                noH = False
            else:
                Hi += g_qc[g] * np.sqrt(omegaq[q] * omegac[c]) * (aqDagList[q] + aqList[q]) * (acList[c] + acDagList[c])
    for g in g_qq:
        q0, q1 = int(g[1]), int(g[3])
        if q0 < qNum and q1 < qNum:
            Hi += g_qq[g] * np.sqrt(omegaq[q0] *  omegaq[q1]) * (aqDagList[q0] + aqList[q0]) * (aqList[q1] + aqDagList[q1])
    return H0, Hi

升降算符，粒子数算符

In [52]:
energyLevel = 3
bitNum = 3
a, aDag = destroy(energyLevel), create(energyLevel)
I = qeye(energyLevel)
IenergyLevel = tensorOperator(energyLevel, I, 0, bitNum)
aqList, aqDagList = [], []
acList, acDagList = [], []
sxList, syList, szList = [], [], []
for b in range(bitNum):
    if b % 2 == 0:
        aq = tensorOperator(energyLevel, a, b, bitNum)
        aqDag = tensorOperator(energyLevel, aDag, b, bitNum)
        aqList.append(aq)
        aqDagList.append(aqDag)
        sxList.append(aq + aqDag)
        syList.append(1j * (aqDag - aq))
        szList.append(IenergyLevel - 2 * aqDag * aq)
    else:
        acList.append(tensorOperator(energyLevel, a, b, bitNum))
        acDagList.append(tensorOperator(energyLevel, aDag, b, bitNum))

# 本征态

能级顺序

In [53]:
energy_info = {'dim':[energyLevel] * bitNum, 'exci_num': energyLevel * bitNum, 'bas_list':[], 'bas_name_list':[]}

for bas in state_number_enumerate(energy_info['dim'], excitations=energy_info['exci_num']):
    energy_info['bas_list'].append(state_number_qobj(energy_info['dim'], bas))
    energy_info['bas_name_list'].append(''.join(map(str,bas)))
energy_info['bas_num'] = len(energy_info['bas_list'])

print(energy_info['bas_name_list'])

['000', '001', '002', '010', '011', '012', '020', '021', '022', '100', '101', '102', '110', '111', '112', '120', '121', '122', '200', '201', '202', '210', '211', '212', '220', '221', '222']


计算本征态并按顺序排列

In [54]:
def eigensolve(H0, H):
    ei_states = H.eigenstates()
    ei_energy = ei_states[0]
    ei_vector = ei_states[1]
    
    ei_states0 = H0.eigenstates()
    ei_energy0 = ei_states0[0]
    ei_vector0 = ei_states0[1]
    
    states_order = ei_vector.copy()
    states0_order = ei_vector.copy()
    energy_order = ei_energy.copy()
    energy0_order = ei_energy.copy()
    for n, vector in enumerate(ei_vector0):
        try:
            index = energy_info['bas_list'].index(vector)
            states_order[index] = ei_vector[n]
            states0_order[index] = ei_vector0[n]
            energy_order[index] = ei_energy[n]
            energy0_order[index] = ei_energy0[n]            
        except:
            pass
    return states_order, states0_order, energy_order, energy0_order

计算zz耦合

In [55]:
def zzcoupling(parameter):
    g_qc = parameter['gqc']
    g_qq = parameter['gqq']
    bitNum = parameter['bitNum']
    omegaq = parameter['omegaq']
    omegac = parameter['omegac']
    
    H0, Hi = H(g_qc, g_qq, omegaq, omegac, (bitNum + 1) // 2, (bitNum - 1) // 2)
    _, _, energy, _ = eigensolve(H0, H0 + Hi)
    zeta = energy[state_number_index(energy_info['dim'], [1, 0, 1])] - energy[state_number_index(energy_info['dim'], [1, 0, 0])] - \
            energy[state_number_index(energy_info['dim'], [0, 0, 1])] + energy[state_number_index(energy_info['dim'], [0, 0, 0])]
    return zeta

扫一个$\omega_c,\omega_2$的图

In [56]:
omega2List = np.arange(4, 6, 2 / 50)
omegacList = np.arange(4, 8, 4 / 50)
zeta_list = []
parameters = []
for omega2 in omega2List:
    for omegac in omegacList:
        parameter = {'gqc' : g_qc, 'gqq' : g_qq, 'bitNum' : bitNum, 
                    'omegaq' : np.array([5, omega2]) * 2 * np.pi, 'omegac' : np.array([omegac]) * 2 * np.pi}
        parameters.append(parameter)
        zeta_list.append(np.abs(zzcoupling(parameter)))
# zeta_list = parallel_map(zzcoupling, parameters, progress_bar=True, num_cpus=4)
zeta_list = np.array([zeta_list]).reshape(len(omega2List), len(omegacList))

plot

In [57]:
xx,yy = np.meshgrid(omega2List, omegacList)
font2 = {'family' : 'Times New Roman', 'weight' : 'normal', 'size' : 20}
fig, ax = plt.subplots()
c = ax.pcolormesh(xx, yy, np.log10(zeta_list.T), cmap='jet')
# c = ax.pcolormesh(xx, yy, zeta_list.T, cmap='jet')
cb=plt.colorbar(c, ax=ax)
cb.set_label('$log_{10}(\zeta)$',fontdict=font2) 

plt.xlabel('$\omega_2$',font2)
plt.ylabel('$\omega_c$',font2)
plt.savefig('xi.pdf', dpi=300)

# 单比特门分析

驱动脉冲\
$\Omega_x=\frac{A}{2}\left(1-\cos(\frac{2\pi t}{t_g})\right)$\
$\Omega_y=-\frac{\lambda\dot{\Omega}_x}{\alpha}$\
$I=\cos\phi,Q=\sin\phi$\
$\Omega(t)=\left(\Omega_xI+\Omega_yQ\right)\cos((\omega_d+\Delta)t)$

In [58]:
def drive_pulseX0(t, args):  
    tg = args['gate time']
    amp = args['q0 amp']
    w_d = args['q0 drive frequency']
    alpha = anharm['q0']
    detune = args['q0 detune']
    lambda0 = args['drag weight']
    phi = args['q0 phi']
    I = np.cos(phi)
    Q = np.sin(phi)
    X0 = amp * (1 - np.cos(2 * np.pi * t / tg)) / 2 
    Y0 = -amp * np.pi / tg * np.sin(2 * np.pi * t / tg) / alpha * lambda0
    X = (X0 * I + Y0 * Q) * np.cos((w_d + detune) * t) 
    return X

def drive_pulseY0(t, args):  
    tg = args['gate time']
    amp = args['q0 amp']
    w_d = args['q0 drive frequency']
    alpha = anharm['q0']
    detune = args['q0 detune']
    lambda0 = args['drag weight']
    phi = args['q0 phi']
    I = np.cos(phi)
    Q = np.sin(phi)
    X0 = amp * (1 - np.cos(2 * np.pi * t / tg)) / 2 
    Y0 = -amp * np.pi / tg * np.sin(2 * np.pi * t / tg) / alpha * lambda0
    X = (Y0 * I - X0 * Q) * np.cos((w_d + detune) * t) 
    return X

def drive_pulseX1(t, args):  
    tg = args['gate time']
    amp = args['q1 amp']
    w_d = args['q1 drive frequency']
    alpha = anharm['q1']
    detune = args['q1 detune']
    lambda1 = args['drag weight']
    phi = args['q1 phi']
    I = np.cos(phi)
    Q = np.sin(phi)
    X0 = amp * (1 - np.cos(2 * np.pi * t / tg)) / 2 
    Y0 = -amp * np.pi / tg * np.sin(2 * np.pi * t / tg) / alpha * lambda1
    X = (X0 * I + Y0 * Q) * np.cos((w_d + detune) * t) 
    return X

def drive_pulseY1(t, args):  
    tg = args['gate time']
    amp = args['q1 amp']
    w_d = args['q1 drive frequency']
    alpha = anharm['q1']
    detune = args['q1 detune']
    lambda1 = args['drag weight']
    phi = args['q1 phi']
    I = np.cos(phi)
    Q = np.sin(phi)
    X0 = amp * (1 - np.cos(2 * np.pi * t / tg)) / 2 
    Y0 = -amp * np.pi / tg * np.sin(2 * np.pi * t / tg) / alpha * lambda1
    X = (Y0 * I - X0 * Q) * np.cos((w_d + detune) * t) 
    return X

保真度公式
$F=\frac{\text{tr}\left(U_\text{eff}^\dagger U_\text{eff}\right)+\left\lvert\text{tr}\left(U^\dagger U_\text{eff}\right)\right\rvert^2}{d(d+1)}$

In [59]:
def Fidelity_X(U):
    d = U.data.shape[0]
    f = lambda phi : -((np.trace(np.dot(U.dag(), U)) + 
                        np.abs(np.trace(np.dot(rz(-phi) * U.dag(), sigmax()))) ** 2) / (d * (d + 1)))
    res = minimize_scalar(f, method='brent')
    return res.fun

哈密顿量演化

In [60]:
def evolution_q0(pulse_paras, pulse_const, H, states, n):
    H_x0 = [sxList[0], drive_pulseX0]
    H_y0 = [syList[0], drive_pulseY0]
    Ht = [H, H_x0, H_y0]
    args = dict()

    args['gate time'] = pulse_const[0] #  [tg, wd0, wd1, drag]
    args['q0 drive frequency'] = pulse_const[1]
    args['drag weight'] = pulse_const[3]

    args['q0 amp'] = pulse_paras[1]
    args['q0 detune'] = pulse_paras[0]
    args['q0 phi'] = -pulse_paras[0] * args['gate time'] / 2

    tList = np.arange(0, args['gate time'], 30 / 300)
    U_full = propagator(Ht, tList, args=args)[-1]
    U=np.zeros([2, 2], dtype='complex128')
    for i in range(2):
        for j in range(2):
            U[i][j] = (states[state_number_index(energy_info['dim'], [n, 0, i])].dag() * U_full * 
                        states[state_number_index(energy_info['dim'], [n, 0, j])]).full()[0][0]
    F = -Fidelity_X(Qobj(U))
    error = 1 - F
    return np.real(error)

def evolution_q1(pulse_paras, pulse_const, H, states, n):
    H_x1 = [sxList[1], drive_pulseX1]
    H_y1 = [syList[1], drive_pulseY1]
    Ht = [H, H_x1, H_y1]
    args = dict()

    args['gate time'] = pulse_const[0] #  [tg, wd0, wd1, drag]
    args['q1 drive frequency'] = pulse_const[2]
    args['drag weight'] = pulse_const[3]

    args['q1 amp'] = pulse_paras[1]
    args['q1 detune'] = pulse_paras[0]
    args['q1 phi'] = -pulse_paras[0] * args['gate time'] / 2

    tList = np.arange(0, args['gate time'], 300)
    U_full = propagator(Ht, tList, args=args)[-1]
    U=np.zeros([2,2], dtype='complex128')
    for i in range(2):
        for j in range(2):
            U[i][j] = (states[state_number_index(energy_info['dim'], [i, 0, n])].dag() * U_full * 
                        states[state_number_index(energy_info['dim'], [j, 0, n])]).full()[0][0]
    F = -Fidelity_X(Qobj(U))
    error = 1 - F
    return np.real(error)

def evolution_q0q1(pulse_paras1, pulse_paras2, pulse_const, H, states, n):
    H_x0 = [sxList[0], drive_pulseX0]
    H_y0 = [syList[0], drive_pulseY0]
    H_x1 = [sxList[1], drive_pulseX1]
    H_y1 = [syList[1], drive_pulseY1]
    Ht = [H, H_x0, H_y0, H_x1 ,H_y1]

    args = dict()

    args['gate time'] = pulse_const[0] #  [tg, wd0, wd1, drag]
    args['q0 drive frequency'] = pulse_const[1]
    args['q1 drive frequency'] = pulse_const[2]
    args['drag weight'] = pulse_const[3]

    args['q0 detune'] = pulse_paras1[0]    
    args['q0 amp'] = pulse_paras1[1]
    args['q0 phi'] = -pulse_paras1[0] * args['gate time'] / 2
    args['q1 detune'] = pulse_paras2[0]
    args['q1 amp'] = pulse_paras2[1]
    args['q1 phi'] = -pulse_paras2[0] * args['gate time'] / 2

    tList = np.arange(0, args['gate time'], 300)
    U_full=propagator(Ht, tList, args = args)[-1]
    U=np.zeros([2,2],dtype='complex128')
    for i in range(2):
        for j in range(2):
            U[i][j]=(states[state_number_index(energy_info['dim'], [1, 0, n])].dag() * U_full *
            states[state_number_index(energy_info['dim'], [1, 0, n])]).full()[0][0]
    F = -Fidelity_X(Qobj(U))
    error = 1 - F
    return np.real(error), U

校准紧邻比特置$|0\rangle$的参数，再用此参数做近邻比特置$|1\rangle$的演化，算误差

In [7]:
def par_X1(parameter):    
    g_qc = parameter['gqc']
    g_qq = parameter['gqq']
    bitNum = parameter['bitNum']
    omegaq = parameter['omegaq']
    omegac = parameter['omegac']
    tg = parameter['gate time']
    wd0 = parameter['q0 drive frequency']
    wd1 = parameter['q1 drive frequency']
    drag = parameter['drag weight']
    pulseConst = [tg, wd0, wd1, drag]

    H0, Hi = H(g_qc, g_qq, omegaq, omegac, (bitNum + 1) // 2, (bitNum - 1) // 2)
    states, _, energy, energy0 = eigensolve(H0, H0 + Hi)
    detune = (energy[state_number_index(energy_info['dim'], [1, 0, 1])] - \
            energy[state_number_index(energy_info['dim'], [1, 0, 0])] - \
            energy0[state_number_index(energy_info['dim'], [1, 0, 1])] + \
            energy0[state_number_index(energy_info['dim'], [1, 0, 0])])
    xIni = [detune, 0.21]
    
    bounds = ((xIni[0] - 10e-3 * 2 * np.pi, xIni[0] + 10e-3 * 2 * np.pi), (xIni[1] - 0.07, xIni[1] + 0.07))
    result0 = minimize(evolution_q0, xIni, args=(pulseConst, H0 + Hi, states, 0), bounds=bounds, method='SLSQP', options={'ftol' : 1e-06})
    err_0 = result0.fun
    err_1 = evolution_q0(result0.x, pulseConst, H0 + Hi, states, 1)
    return err_0, err_1


扫图

In [8]:
omega0 = 5
omega1List = np.arange(4, 6, 2 / 5)
omegacList = np.arange(4, 8, 4 / 5)
err0_list = []
err1_list = []
parameters = []
bitNum = 3

for omega1 in omega1List:
    for omegac in omegacList:
        parameter = {'gqc' : g_qc, 'gqq' : g_qq, 'bitNum' : bitNum, 
                    'omegaq' : np.array([omega0, omega1]) * 2 * np.pi, 'omegac' : np.array([omegac]) * 2 * np.pi,
                    'gate time' : 30, 'q0 drive frequency' : omega0 * 2 * np.pi, 'q1 drive frequency' : omega1 * 2 * np.pi, 'drag weight' : 0.5}
        parameters.append(parameter)
        error_0, error_1 = par_X1(parameter)
        print(omega1, omegac, error_0, error_1)
        err0_list.append(error_0)
        err1_list.append(error_1)

# result1 = parallel_map(par_X1, parameter, progress_bar=True,num_cpus=22)     
err0_list = np.array([err0_list]).reshape(len(omega1List), len(omegacList))
err1_list = np.array([err1_list]).reshape(len(omega1List), len(omegacList))

NameError: name 'H' is not defined

plot

In [ ]:
xx,yy = np.meshgrid(omega1List, omegacList)
font2 = {'family' : 'Times New Roman', 'weight' : 'normal', 'size' : 20}
fig, ax = plt.subplots()
c = ax.pcolormesh(xx, yy, np.log10(error0_list.T), cmap='jet')
cb=plt.colorbar(c, ax=ax)
cb.set_label('$log_{10}(\epsilon)$',fontdict=font2) 

plt.xlabel('$\omega_2$',font2)
plt.ylabel('$\omega_c$',font2)
plt.savefig('erro.pdf', dpi=300)

xx,yy = np.meshgrid(omega1List, omegacList)
font2 = {'family' : 'Times New Roman', 'weight' : 'normal', 'size' : 20}
fig, ax = plt.subplots()
c = ax.pcolormesh(xx, yy, np.log10(error1_list.T), cmap='jet')
cb=plt.colorbar(c, ax=ax)
cb.set_label('$log_{10}(\epsilon)$',fontdict=font2) 

plt.xlabel('$\omega_2$',font2)
plt.ylabel('$\omega_c$',font2)
plt.savefig('erro.pdf', dpi=300)